<a href="https://colab.research.google.com/github/jebin-D1/fine_tune0_1/blob/main/fine_tune0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
from datasets import Dataset

# Load JSONL file
file_path = "prompt_02_.jsonl"

data = []
with open(file_path, "r") as f:
    for line in f:
        data.append(json.loads(line))  # Each line is its own dict

print("Sample:", data[0])

Sample: {'input': 'What should I do? I forgot my password?', 'output': "No worries at all! Please visit our login page and select 'Forgot Password' to reset your password. You will immediately receive an email with a secure link to create a new one. Please check your spam or junk folder if you haven't seen the email in a few minutes."}


In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel

model_name = "unsloth/llama-3-8b-bnb-4bit"

max_seq_length = 2048
dtype = None  # Auto detection

# Load model + tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.10: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [ ]:
# Convert dataset into text format
def format_prompt(example):
    return f"### Input: {example['input']}\n### Output: {example['output']}<|endoftext|>"

formatted_data = [format_prompt(item) for item in data]
dataset = Dataset.from_dict({"text": formatted_data})

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.8.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/7 [00:00<?, ? examples/s]

In [ ]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7 | Num Epochs = 3 | Total steps = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 167,772,160 of 8,198,033,408 (2.05% trained)


Step,Training Loss,entropy


In [ ]:
FastLanguageModel.for_inference(model)

# Your test sample
test_input = "What should I do? I forgot my password?"
prompt = f"### Input: {test_input}\n### Output:"

# Tokenize
batch = tokenizer(
    prompt,
    return_tensors="pt",
    add_special_tokens=False,
)

# Ensure pad token exists
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Send to model device
batch = {k: v.to(model.device) for k, v in batch.items()}

# Generate
with torch.inference_mode():
    output_ids = model.generate(
        **batch,
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        use_cache=True,
    )

# Decode and clean
decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
answer = decoded.split("### Output:", 1)[-1].strip()

print({"input": test_input, "output": answer})

{'input': 'What should I do? I forgot my password?', 'output': "No worries! Here is a list of recommended next steps:\n\n    - First, try resetting your password using the email address associated with your account. If you can't remember the email address, you may need to contact the website administrator or customer service for assistance.\n    - If resetting your password doesn't work, try logging in using a different browser or clearing your browser's cache and cookies. This can sometimes help resolve issues with logging in.\n    - Finally, if you still can't log in, consider creating a new account with the same email address. This way, you can easily access your information without having to worry about remembering your old password.\n    \n    In conclusion, there are several potential solutions to this problem, depending on the specific circumstances. However, it's always best to start by trying to reset your password and then gradually work your way up to more advanced solutions

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
if gguf_files:
    gguf_file = os.path.join("gguf_model", gguf_files[0])
    print(f"Downloading: {gguf_file}")
    files.download(gguf_file)

FileNotFoundError: [Errno 2] No such file or directory: 'gguf_model'